<a href="https://colab.research.google.com/github/ME-UoP-ME421/Kalman-FTS/blob/main/LSTM_RNN_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##LSTM RNN Model to predict human driver behavior

Labels: Acceleration, Deceleration, Gear

Feature: Axle Torque

In [37]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


In [20]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
#!ls "/content/drive/My Drive/FYP"
path = "/content/drive/My Drive/FYP/driver_dataset_3.csv"
df = pd.read_csv(path, usecols = [1, 5, 6, 7], engine = 'python')

In [55]:
df

,<AxlTrq>(1),<AccelFdbk>,<DecelFdbk>,<GearFdbk>
0,0.000000e+00,0.000000,0.0,0
1,-1.201350e-39,0.000000,0.0,0
2,-1.201350e-39,0.000000,0.0,0
3,2.062970e-14,0.000000,0.0,0
4,2.062970e-14,0.000000,0.0,0
...,...,...,...,...
247771,-4.333370e-15,0.015416,0.0,1
247772,-6.693260e-15,0.010857,0.0,1
247773,-7.005880e-15,0.006298,0.0,1
247774,-4.339190e-15,0.001739,0.0,1


In [62]:
# Split the data into the inputs and outputs
features = df.iloc[:, 1:4]    #acceleration, deceleration and gear are input features
target = df.iloc[:, 0]        #axle torque is target output variable

In [ ]:
features

In [ ]:
target

In [70]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

#Normalize the data
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(features)

#reshape the data for LSTM input [sample, time step, features]
time_steps = 1      #increase for sequence prediction task
reshaped_features = np.reshape(scaled_features, (scaled_features.shape[0], time_steps, scaled_features.shape[1]))

#split data into training and test
feature_train, feature_test, target_train, target_test = train_test_split(reshaped_features, target, test_size = 0.2, random_state = 42)

#build LSTM RNN model
model = Sequential()
model.add(LSTM(64, input_shape = (feature_train.shape[1], feature_train.shape[2])))
model.add(Dense(1))


In [80]:
#compile the model
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [82]:
#train the model

model.fit(feature_train, target_train, epochs = 4, batch_size = 1, verbose = 2)

Epoch 1/4
198220/198220 - 350s - loss: 1.4345 - 350s/epoch - 2ms/step
Epoch 2/4
198220/198220 - 350s - loss: 1.4337 - 350s/epoch - 2ms/step
Epoch 3/4
198220/198220 - 348s - loss: 1.4343 - 348s/epoch - 2ms/step
Epoch 4/4
198220/198220 - 342s - loss: 1.4348 - 342s/epoch - 2ms/step


In [83]:
model.fit(feature_train, target_train, epochs = 20, batch_size = 1, verbose = 2)

Epoch 1/20
198220/198220 - 342s - loss: 1.4346 - 342s/epoch - 2ms/step
Epoch 2/20
198220/198220 - 343s - loss: 1.4350 - 343s/epoch - 2ms/step
Epoch 3/20
198220/198220 - 344s - loss: 1.4345 - 344s/epoch - 2ms/step
Epoch 4/20
198220/198220 - 344s - loss: 1.4345 - 344s/epoch - 2ms/step
Epoch 5/20
198220/198220 - 342s - loss: 1.4343 - 342s/epoch - 2ms/step
Epoch 6/20
198220/198220 - 342s - loss: 1.4345 - 342s/epoch - 2ms/step
Epoch 7/20
198220/198220 - 344s - loss: 1.4354 - 344s/epoch - 2ms/step
Epoch 8/20
198220/198220 - 342s - loss: 1.4344 - 342s/epoch - 2ms/step
Epoch 9/20
198220/198220 - 342s - loss: 1.4347 - 342s/epoch - 2ms/step
Epoch 10/20
198220/198220 - 342s - loss: 1.4343 - 342s/epoch - 2ms/step
Epoch 11/20
198220/198220 - 343s - loss: 1.4340 - 343s/epoch - 2ms/step
Epoch 12/20
198220/198220 - 344s - loss: 1.4348 - 344s/epoch - 2ms/step
Epoch 13/20
198220/198220 - 343s - loss: 1.4344 - 343s/epoch - 2ms/step
Epoch 14/20
198220/198220 - 344s - loss: 1.4337 - 344s/epoch - 2ms/step
E

In [ ]:
model.fit(feature_train, target_train, epochs = 100, batch_size = 1, verbose = 2)

# Evaluate the model on the test set
loss = model.evaluate(feature_test, target_test)
print("Test loss:", loss)

# Save the trained model (optional)
model.save('/content/drive/My Drive/FYP/lstm_rnn_model1.h5')

Epoch 1/100
198220/198220 - 349s - loss: 1.4341 - 349s/epoch - 2ms/step
Epoch 2/100
198220/198220 - 346s - loss: 1.4337 - 346s/epoch - 2ms/step
Epoch 3/100


In [88]:
from keras.models import load_model

# Load the saved LSTM RNN model
saved_model_path = '/content/drive/My Drive/FYP/lstm_rnn_model.h5'
model = load_model(saved_model_path)

# Make predictions on the test data
predictions = model.predict(reshaped_features)

# Inverse transform each feature of the predictions back to their original scale
for i in range(predictions.shape[1]):
    predictions[:, i] = scaler.inverse_transform(predictions[:, i].reshape(-1, 1)).flatten()

# Print the predictions
print("Predictions:")
print(predictions)

7743/7743 [==============================] - 12s 2ms/step


ValueError: ignored